### Make sure we can used GPU

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


### Import libraries

In [2]:
import numpy as np
import os
from keras.datasets import mnist
from keras.utils import to_categorical
from keras import models
from keras import layers
import matplotlib.pyplot as plt

Using TensorFlow backend.


### Download data

In [3]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


### Create NN
---
* Input tenzor  shape = (28, 28, 1) #Height, Width, Channel Color
* Size templates - usually 3 x 3 or 5 x 5

In [0]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1))) # Asix Deep 1(gray)
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))

Свертка работает методом скользящего окна: она двигает окно размером(3х3/5х5) по трехмерной входной карте признаков, останавливаесь на каждой возможно позиции и извлекая трехмерный шаблон окружающих признаков.

Первый сверточный слой принимает карту признаков с разрмером(28, 28, 1) и **выводит карту признаков размеров(26, 26, 32)**:
* Он вычисляет 32 фильтра по входным данным
* Каждый из этих 32 фильтров содержит сетку 26 х 26 значений - **Карту ответов** - фильтра на входные данные определяющие ответ этого шаблона фильтра на разных участках входных данных
* **Карта признаков** - каждое измерение на оси глубины - это признак(фильтр), а двумерный тензнор output[ :, :, n] - это двумерная пространственная карта ответов этого фильтра на входных данных. 


**MaxPooling2D - аргессивное уменьшение разрешения карты признаков данной операции.**
Уменьшение разрешения используется для уменьшения количества коэф в карте признаков для обработки

Цель слоя – уменьшение размерности карт предыдущего слоя. Если на предыдущей операции свертки уже были выявлены некоторые признаки, то для дальнейшей обработки настолько подробное изображение уже не нужно, и оно уплотняется до менее подробного. К тому же фильтрация уже ненужных деталей помогает не переобучаться. 
В процессе сканирования ядром подвыборочного слоя (фильтром) карты предыдущего слоя, сканирующее ядро не пересекается в отличие от сверточного слоя. Обычно, каждая карта имеет ядро размером 2x2, что позволяет уменьшить предыдущие карты сверточного слоя в 2 раза. Вся карта признаков разделяется на ячейки 2х2 элемента, из которых выбираются максимальные по значению.

**MaxPooling – выбор максимального**
![](https://habrastorage.org/webt/0u/ji/tm/0ujitma2xn_ndxqswj5s31je2am.png)

Окно размером ядра K(3x3) проходит с заданным шагом (обычно 1) все изображение I, на каждом шаге поэлементно умножаем содержимое окна на ядро K, результат суммируется и записывается в матрицу результата:
![asa](https://habrastorage.org/webt/v9/k2/kc/v9k2kc8ng4nrhryunr3wr6l5brg.png)

![image](https://habrastorage.org/webt/o0/zh/rz/o0zhrzr_ml2tgsfmvl-mcrxjmbq.gif)

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 128)         73856     
Total params: 92,672
Trainable params: 92,672
Non-trainable params: 0
_________________________________________________________________


### Architecture Network
* All layers output 3D Tenzor with shape(height, width, channel)
* Shapes Compressed Every Layer
* Number channels controlled first arg in Conv2D

### Next Step we need give last input layer(3, 3, 128) in classification network. Classification works with 1D array, so we need turn our 3D in 1D.

In [0]:
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

### Flatten Layer converts output(3, 3, 128)

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 128)         73856     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               147584    
__________

In [0]:
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels  = to_categorical(test_labels)

In [11]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 13s 217us/step - loss: 0.1393 - acc: 0.9555
Epoch 2/5
60000/60000 [==============================] - 11s 190us/step - loss: 0.0391 - acc: 0.9879
Epoch 3/5
60000/60000 [==============================] - 12s 195us/step - loss: 0.0266 - acc: 0.9919
Epoch 4/5
60000/60000 [==============================] - 12s 197us/step - loss: 0.0209 - acc: 0.9940
Epoch 5/5
60000/60000 [==============================] - 12s 195us/step - loss: 0.0156 - acc: 0.9953


In [12]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 1s 117us/step


In [13]:
print('Accuracy', test_acc * 100, '%')

Accuracy 99.22999999999999 %


### Change Colab Directory

In [0]:
pwd

'/content'

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd drive/My Drive/DeepLearning/Colab Notebooks

/content/drive/My Drive/DeepLearning/Colab Notebooks


### Save Our Model

In [0]:
pwd

'/content/drive/My Drive/DeepLearning/Colab Notebooks'

In [0]:
path = 'model_weight/'

In [0]:
L2_model.save(os.path.join(path, 'L2_model.h5'))

### Load Model 

In [0]:
from keras.models import load_model
model_L2 = load_model(os.path.join(path, 'L2_model.h5'))

### Download Files On Computer

In [0]:
from google.colab import files
files.download("model_weight/L2_model.h5")

In [0]:
x_train.shape, x_test.shape

((8982,), (2246,))